# SIC PROJECT



In [107]:
#requirements and dependencies
#check if the requirements are installed
requirements = ["tensorflow", "numpy", "cv2", "tqdm", "matplotlib", "tensorboard", "tensorflow_hub", "sklearn"]
#if not, quit
for requirement in requirements:
    try:
        __import__(requirement)
    except ImportError:
        print("You need to install the following package: " + requirement)
        print("You can do this by running: pip install " + requirement)
        quit()
print("All requirements are installed")

All requirements are installed


In [108]:
#import packages
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import datetime
from sklearn.model_selection import train_test_split

import tensorboard
%load_ext tensorboard
print("success")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
success


In [109]:
#check metal gpu
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

/device:GPU:0
Num GPUs Available:  1


2023-07-31 03:00:11.557497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-31 03:00:11.557518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [110]:
#load data
def load_data(path, IMG_SIZE=224):
    """
    Load image data from directory, resize and normalize it.
    return tuple `(images, labels)`.
    """
    categories = os.listdir(path)
    categories.remove('.DS_Store')
    data = []
    labels = []
    for category in tqdm.tqdm(categories):
        folder_path = os.path.join(path, category)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                labels.append(categories.index(category))

    return np.array(data), tf.keras.utils.to_categorical(labels)


In [111]:
#built from scratch model
def get_model(IMG_SIZE = 224, NUM_CLASSES = 3):
    
    #model architecture
    model = tf.keras.models.Sequential([

       tf.keras.layers.Conv2D(
           64, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)
       ),

       tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

       tf.keras.layers.Dropout(0.25),

       tf.keras.layers.Conv2D(
           128, (3, 3), activation="relu"),

       tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

       tf.keras.layers.Dropout(0.25),

       tf.keras.layers.Flatten(),

       tf.keras.layers.Dense(128, activation="relu"),

       tf.keras.layers.Dropout(0.25),
       ##

       tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")

    ])

    # Compile Model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [114]:
#load data 
train_data, train_labels = load_data('Train')
test_data, test_labels = load_data('Test')

#load model
model = get_model()

#some parameters
EPOCHS = 1
IMG_SIZE = 224
BATCH_SIZE = 32

#for tensorboard
log_dir = "logs/fit/" + str(len(os.listdir('logs/fit')))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,  histogram_freq=1)

#train the model and validate the model #TODO: split the train data into train and validation, DON'T USE TEST DATA FOR VALIDATION*
history = model.fit(train_data, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[tensorboard_callback])
#evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels, callbacks=[tensorboard_callback])

100%|██████████| 3/3 [00:00<00:00,  6.44it/s]


Train on 2699 samples


2023-07-31 03:03:48.984787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-31 03:03:51.264582: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_16/kernel/Assign' id:44172 op device:{requested: '', assigned: ''} def:{{{node conv2d_16/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_16/kernel, conv2d_16/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-31 03:03:57.316789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-31 03:04:01.416362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimize

  32/2699 [..............................] - ETA: 14:52 - loss: 44.8403 - accuracy: 0.4062

2023-07-31 03:04:14.401850: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-07-31 03:04:14.401872: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


  64/2699 [..............................] - ETA: 7:53 - loss: 14315.8733 - accuracy: 0.3594

2023-07-31 03:04:15.137585: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2023-07-31 03:04:15.140991: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2023-07-31 03:04:15.143162: I tensorflow/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/fit/20230731-030335/plugins/profile/2023_07_31_03_04_15/Mohammeds-MBP.xplane.pb


2699/2699 [==============================] - 35s 13ms/sample - loss: 1081.9914 - accuracy: 0.3583


In [ ]:
#lunch tensorboard to plot the results and the graphs
print("model accuracy: " + str(test_acc))
print("allah y3een")
%tensorboard --logdir=logs --port=6969